https://adventofcode.com/2020

In [1]:
import collections
import functools
import itertools
import operator
import re

import pandas as pd
import numpy as np

In [2]:
# 1
with open('input1.txt', 'r') as f:
    nums = [int(x) for x in f.readlines()]
seen = set()
for x in nums:
    y = 2020-x
    if y in seen:
        print(x, y, x*y)
    seen.add(x)

319 1701 542619


In [3]:
seen = set()
for i, x in enumerate(nums):
    for y in nums[i+1:]:
        z = 2020-x-y
        if z in seen:
            print(x, y, z, x*y*z)
    seen.add(x)

43 527 1450 32858450


In [4]:
# 2
with open('input2.txt', 'r') as f:
    inps = f.readlines()

inp_regex = '^(\d+)-(\d+) ([a-z]): ([a-z]*)'
result = 0
for inp in inps:
    mn, mx, char, pas = re.match(inp_regex, inp).groups()
    mn, mx = int(mn), int(mx)
    result += mn <= collections.Counter(pas)[char] <= mx
result

524

In [5]:
result = 0
for inp in inps:
    i, j, char, pas = re.match(inp_regex, inp).groups()
    i, j, N = int(i)-1, int(j)-1, len(pas)
    result += (i < N and pas[i] == char) ^ (j < N and pas[j] == char)
result

485

In [6]:
# 3
with open('input3.txt', 'r') as f:
    grid = [[1*(c=='#') for c in l] for l in f.read().splitlines()]
N, M = len(grid), len(grid[0])
result = sum(grid[i][(i*3) % M] for i in range(N))
result

299

In [7]:
results = []
for r, d in [(1,1),(3,1),(5,1),(7,1),(1,2)]:
    results.append(sum(grid[i][(i*r//d) % M] for i in range(0,N,d)))
results, functools.reduce(operator.mul, results)

([67, 299, 67, 71, 38], 3621285278)

In [8]:
# 4
with open('input4.txt', 'r') as f:
    passports = [p.replace('\n', ' ').split() for p in 
              f.read().split('\n\n')]
target_fields = {'byr', 'iyr', 'eyr', 'hgt',
                 'hcl', 'ecl', 'pid'} #, 'cid'}
result = 0
for p in passports:
    fields = dict(e.split(':') for e in p)
    result += all(t in fields for t in target_fields)
result

254

In [9]:
def is_digit_in_incl_range(s, mn, mx):
    try:
        return mn <= int(s) <= mx
    except:
        return False
    
def is_valid_height(s):
    if len(s) < 3:
        return False
    n, t = s[:-2], s[-2:]
    if t == 'cm':
        return is_digit_in_incl_range(n, 150, 193)
    elif t == 'in':
        return is_digit_in_incl_range(n, 59, 76)
    return False

eye_cols = ['amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth']

result = 0
for p in passports:
    fields = dict(e.split(':') for e in p)
    valid = all(t in fields for t in target_fields)
    valid = valid and is_digit_in_incl_range(fields['byr'], 1920, 2002)
    valid = valid and is_digit_in_incl_range(fields['iyr'], 2010, 2020)
    valid = valid and is_digit_in_incl_range(fields['eyr'], 2020, 2030)
    valid = valid and is_valid_height(fields['hgt'])
    valid = valid and re.match('^#[0-9a-f]{6}$', fields['hcl']) is not None
    valid = valid and fields['ecl'] in eye_cols
    valid = valid and re.match('^[0-9]{9}$', fields['pid']) is not None
    result += valid
result

184

In [10]:
# 5
with open('input5.txt', 'r') as f:
    inp = f.read().splitlines()
    
binary = [''.join(['0','1'][c in 'BR'] for c in l) for l in inp]
ids = [8*int(b[:-3], 2) + int(b[-3:], 2) for b in binary]
max(ids)

935

In [11]:
ids = sorted(ids)
miss = [i for i in range(len(ids)-1) if not ids[i]+1 == ids[i+1]][0]
ids[miss]+1

743

In [12]:
# 6
with open('input6.txt', 'r') as f:
    groups = [g.split('\n') for g in f.read().split('\n\n')]
    
sum(len(set(''.join(g))) for g in groups)

6782

In [13]:
sum(len(g)==v for g in groups 
    for v in collections.Counter(''.join(g)).values())

3596

In [14]:
# 7
with open('input7.txt', 'r') as f:
    raw_rules = f.read().splitlines()

regex = '(\d+) ([a-z\s]+)$'
def parse_rule(rule):
    s = rule.split(' bag')
    try:
        r = dict(re.search(regex, r).groups()[::-1] for r in s[1:-1])
        r = {k:int(v) for k,v in r.items()}
    except Exception as e:
        r = {}
    return s[0], r
rules = dict(parse_rule(r) for r in raw_rules)
dict(list(rules.items())[:5])

{'wavy green': {'posh black': 1, 'faded green': 1, 'wavy red': 4},
 'dotted chartreuse': {'light beige': 1},
 'dark white': {'dotted white': 2},
 'clear aqua': {'posh orange': 4, 'pale blue': 4},
 'faded green': {'plaid orange': 2,
  'drab green': 4,
  'pale aqua': 4,
  'drab coral': 4}}

In [15]:
target = 'shiny gold'

In [16]:
leads_to_target = set([target])
not_to_target = set()

In [17]:
def helper(typ):
    # assumes no loops
    if typ in leads_to_target:
        return True
    if typ in not_to_target:
        return False
    to_target = False
    for styp in rules[typ].keys():
        to_target |= helper(styp)
    if to_target:
        leads_to_target.add(typ)
    else:
        not_to_target.add(typ)
    return to_target
    
for typ in rules:
    helper(typ)

In [18]:
len(leads_to_target)-1

169

In [19]:
def helper(typ):
    # assumes no loops
    return sum((1+helper(styp))*c for styp, c in rules[typ].items())

In [20]:
helper(target)

82372

In [21]:
# 8
with open('input8.txt', 'r') as f:
    operations = [l.split() for l in f.read().splitlines()]

def run_operations(operations):
    seen = [False] * len(operations)
    curr, acc = 0, 0
    loopy = False
    while curr < len(operations):
        if seen[curr]:
            loopy = True
            break
        seen[curr] = True
        op, val = operations[curr]
        val = int(val)
        if op == 'acc':
            acc += val
            curr += 1
        elif op == 'jmp':
            curr += val
        else:
            curr += 1
    return acc, loopy

run_operations(operations)

(1487, True)

In [22]:
# I use brute force here
# A non brute force solution is possible.
# Firstly run backgrounds from the end to see which operations
# will lead to the end. Then run forward and see which operation
# can be changed to map to one of those operations that will lead
# to the end.
for i in range(len(operations)):
    copy = operations.copy()
    op, val = operations[i]
    if op == 'acc':
        continue
    op = 'nop' if op == 'jmp' else 'jmp'
    copy[i] = [op, val]
    acc, loopy = run_operations(copy)
    if not loopy:
        print(i, acc)

469 1607


In [23]:
# 9
with open('input9.txt', 'r') as f:
    nums = [int(x) for x in f.readlines()]

In [24]:
seen = set(nums[:25])
for n in nums[25:]:
    valid = any(n-s in seen for s in seen if s!=n/2)
    if valid:
        seen.add(n)
    else:
        result = n
        break
result

552655238

In [25]:
cumsum = list(itertools.accumulate(nums))
for i in range(len(cumsum)):
    for j in range(i+1, len(cumsum)):
        if cumsum[j] - cumsum[i] == result:
            rng = nums[i+1:j+1]
            print(rng)
            print(*(f(rng) for f in (sum, min, max)))
            

[30484395, 26952678, 25903382, 26433970, 27327951, 29543326, 31321247, 30139557, 31044336, 32989305, 31818413, 34730529, 35979636, 41140695, 35790623, 36286332, 44768863]
552655238 25903382 44768863
[552655238]
552655238 552655238 552655238


In [26]:
25903382 + 44768863

70672245

In [27]:
# 10
with open('input10.txt', 'r') as f:
    jolts = [int(x) for x in f.readlines()]

In [28]:
jolts = [0] + sorted(jolts) + [max(jolts)+3]
diffs = [n-p for p,n in zip(jolts, jolts[1:])]
collections.Counter(diffs)

Counter({1: 70, 3: 35})

In [29]:
70*35

2450

In [30]:
dp = [0] * len(jolts)
dp[0] = 1
for i, jolt in enumerate(jolts[1:], 1):
    num_pths = 0
    for j in range(max(0, i-3),i):
        if jolts[j] >= jolt - 3:
            num_pths += dp[j]
    dp[i] = num_pths

In [31]:
dp[-1]

32396521357312

In [32]:
# 11
# ENUM 0 -> floor, 1 -> empty, 2 -> occupied
enum = {'.': 0, 'L': 1, '#': 2}
renum = {v:k for k,v in enum.items()}
with open('input11.txt', 'r') as f:
    seats = np.array([[enum[c] for c in l] for l in f.read().splitlines()])
    
orig_seats = seats = np.pad(seats, (1, 1))
seats[:4,:4]

array([[0, 0, 0, 0],
       [0, 1, 1, 1],
       [0, 1, 1, 1],
       [0, 1, 1, 0]])

In [33]:
def evolve(seats):
    occ = seats == 2
    num_occ = np.sum([        
        np.roll(occ, (i, j), (0, 1))
        for i in [-1, 0, 1]
        for j in [-1, 0, 1]
    ], 0) - occ
    evolved = seats.copy()
    evolved[(seats == 1) & (num_occ == 0)] = 2
    evolved[(seats == 2) & (num_occ >= 4)] = 1
    return evolved

done = False
while not done:
    evolved = evolve(seats)
    if np.all(evolved == seats):
        done = True
    else:
        seats = evolved
np.sum(seats==2)

2472

In [34]:
# part 2 is a joy..., lets throw the beauty of numpy out the window
# it seems like people attempting APL and J also cried at this point
# a faster implementation would be possible with clever use of
# itertools.accumulate translated to the numpy equivalent
seats = orig_seats

def evolve(seats):
    occ = seats == 2
    def calc(i, j):
        return sum(
            calcd(i, j, id, jd)
            for id in [-1, 0, 1]
            for jd in [-1, 0, 1]
            if not (id==0 and jd==0)
        )
    def calcd(i, j, id, jd):
        i, j = i+id, j+jd
        while True:
            if not (0 <= i < len(seats) and \
                    0 <= j < len(seats[i])):
                # out of bounds
                return 0
            if seats[i][j] == 1:
                # empty
                return 0
            if seats[i][j] == 2:
                # occupied
                return 1
            i += id
            j += jd           
    num_occ = np.array([
        [calc(i, j) for j in range(len(seats[i]))]
        for i in range(len(seats))
    ])
    evolved = seats.copy()
    evolved[(seats == 1) & (num_occ == 0)] = 2
    evolved[(seats == 2) & (num_occ >= 5)] = 1
    return evolved

done = False
while not done:
    evolved = evolve(seats)
    if np.all(evolved == seats):
        done = True
    else:
        seats = evolved
np.sum(seats==2)

2197